In [1]:
import pandas as pd
import datetime as dt
from datetime import datetime
import matplotlib.pyplot as plt
import re
import requests

In [2]:
import requests

url = "https://yahoo-finance15.p.rapidapi.com/api/yahoo/hi/history/cobr.l/1d"

headers = {
    'x-rapidapi-key': "9cd97ba3camsh244cbd2e3e420a4p1e83c0jsne0fdd0787f8e",
    'x-rapidapi-host': "yahoo-finance15.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers)

print(response.text)

In [3]:
string = response.text

In [20]:
string[:1000]

'{\n    "meta": {\n        "currency": "GBp",\n        "symbol": "COBR.L",\n        "exchangeName": "LSE",\n        "instrumentType": "EQUITY",\n        "firstTradeDate": 1542268800,\n        "regularMarketTime": 1628081844,\n        "gmtoffset": 3600,\n        "timezone": "BST",\n        "exchangeTimezoneName": "Europe/London",\n        "regularMarketPrice": 1.736,\n        "chartPreviousClose": 1.75,\n        "priceHint": 4,\n        "dataGranularity": "1d",\n        "range": ""\n    },\n    "items": {\n        "1542268800": {\n            "date": "15-11-2018",\n            "open": 1.67,\n            "high": 1.75,\n            "low": 1.67,\n            "close": 1.75,\n            "adjclose": 1.75\n        },\n        "1542355200": {\n            "date": "16-11-2018",\n            "open": 1.75,\n            "high": 1.75,\n            "low": 1.75,\n            "close": 1.75,\n            "adjclose": 1.75\n        },\n        "1542614400": {\n            "date": "19-11-2018",\n         

获取日期data

In [4]:
def get_dates(string):
    date = re.compile('"date":((.*),)')
    date = date.findall(string)
    date = [i[0][2:12] for i in date]
    date = [datetime.strptime(i, "%d-%m-%Y").strftime("%Y-%m-%d") for i in date]
    return date

In [10]:
date = get_dates(string)
print( date[:5] )

['2018-11-15', '2018-11-16', '2018-11-19', '2018-11-20', '2018-11-21']


获取close价格

In [14]:
def get_close(string):
    close = re.compile('"close":((.*),)')
    close = close.findall(string)
    close = [float(i[1].strip()) for i in close]
    return close

In [15]:
close = get_close(string)

In [18]:
close[:5]

[1.75, 1.75, 1.65, 1.65, 1.65]

In [ ]:
# import re
# date = re.compile('"date":((.*),)')
# print (date.findall(string)[:10])

# dates = date.findall(string)

# s_dates = []
# for i in dates:
#     s_dates.append(i[0][2:12])


# s_dates[:5]

# cobr_dates=[]
# for i in s_dates:
#     cobr_dates.append(datetime.strptime(i, "%d-%m-%Y").strftime("%Y-%m-%d"))

# cobr_dates[:10]

In [ ]:
close = re.compile('"close":((.*),)')
# print(close.findall(string))

In [ ]:
s_close = close.findall(string)

In [ ]:
closes = []
for i in s_close:
    closes.append(i[1].strip())

In [ ]:
cobr_close = [float(i) for i in closes]

In [ ]:
cobr_close[:10]

In [ ]:
print(len(cobr_dates))
print(len(cobr_close))

In [ ]:
adjclose = re.compile('"close":((.*),)')
s_adjclose = adjclose.findall(string)

In [ ]:
adj_closes = []
for i in s_close:
    adj_closes.append(i[1].strip())

In [ ]:
cobr_adjclose = [float(i) for i in adj_closes]

In [ ]:
cobr_adjclose[:10]

In [ ]:
cobr = pd.DataFrame({'dates':cobr_dates, 'close':cobr_close,'adjclose':cobr_adjclose})

In [ ]:
cobr.to_csv(r'/Users/paxton615/Desktop/COBR.csv')

In [ ]:
cobr.columns

In [ ]:
cobr.set_index('dates', drop=True, inplace=True)

In [ ]:
plt.figure(figsize=(16,10))
cobr.close.plot();